<a href="https://colab.research.google.com/github/aleksudan/automated-security-helper-pipeline/blob/main/System_Performance_Monitor_Script_(Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import psutil
import time
import datetime
import csv
import os

# Define the log file name
# This file will store the performance data
LOG_FILE = 'system_performance_log.csv'

def get_system_usage():
    """
    Fetches current CPU and memory usage percentages.

    Returns:
        tuple: A tuple containing:
               - float: CPU usage percentage over the last 1 second.
               - float: Memory usage percentage.
    """
    # psutil.cpu_percent(interval=1) calculates the CPU usage over a 1-second interval.
    cpu_percent = psutil.cpu_percent(interval=1)
    # psutil.virtual_memory() gets statistics about system memory usage.
    mem_info = psutil.virtual_memory()
    # mem_info.percent gives the percentage of used memory.
    mem_percent = mem_info.percent
    return cpu_percent, mem_percent

def log_to_csv(timestamp, cpu, memory):
    """
    Logs the performance data (timestamp, CPU, memory) to a CSV file.

    Args:
        timestamp (str): The formatted current time string.
        cpu (float): The CPU usage percentage.
        memory (float): The memory usage percentage.
    """
    # Check if the log file already exists. This is used to decide whether to write the header row.
    file_exists = os.path.isfile(LOG_FILE)

    # Open the CSV file in append mode ('a'). 'newline=''' prevents extra blank rows in the CSV.
    with open(LOG_FILE, mode='a', newline='') as file:
        # Create a CSV writer object.
        writer = csv.writer(file)
        # If the file did not exist before opening, write the header row.
        if not file_exists:
            writer.writerow(['Timestamp', 'CPU_Usage (%)', 'Memory_Usage (%)'])
        # Write the current data row to the CSV file.
        writer.writerow([timestamp, cpu, memory])

def monitor_system(interval_seconds=5, duration_minutes=None):
    """
    Monitors system usage at specified intervals and logs it to a CSV file.

    Args:
        interval_seconds (int): The time interval between each monitoring sample, in seconds.
                                Defaults to 5 seconds.
        duration_minutes (int, optional): The total duration to monitor the system, in minutes.
                                          If None, the script will monitor indefinitely until stopped
                                          manually (e.g., by pressing Ctrl+C).
                                          Defaults to None.
    """
    print(f"Starting system monitoring. Data will be logged to {LOG_FILE}")
    print(f"Logging interval: {interval_seconds} seconds")
    if duration_minutes:
        print(f"Monitoring duration: {duration_minutes} minutes")
        end_time = time.time() + (duration_minutes * 60) # Calculate the end time if duration is set
    else:
        end_time = float('inf') # Monitor indefinitely if no duration is set

    # Use a try-except block to handle manual interruption (Ctrl+C) gracefully.
    try:
        # The main monitoring loop. It continues as long as the current time is less than the end time.
        while time.time() < end_time:
            # Get the current time and format it as a string.
            current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            # Fetch the current CPU and memory usage.
            cpu, memory = get_system_usage()

            # Print the current usage to the console for immediate feedback.
            print(f"{current_time} - CPU: {cpu:.1f}%, Memory: {memory:.1f}%")
            # Log the data to the CSV file.
            log_to_csv(current_time, cpu, memory)

            # Pause the script for the specified interval before the next sample.
            time.sleep(interval_seconds)

        # This message is printed if the monitoring duration is reached.
        if duration_minutes:
             print(f"Monitoring duration of {duration_minutes} minutes reached. Stopping.")


    except KeyboardInterrupt:
        # This block is executed if the user presses Ctrl+C.
        print("\nMonitoring stopped manually.")
    except Exception as e:
        # This block catches any other unexpected errors.
        print(f"\nAn error occurred: {e}")

# --- How to Run ---
# This block ensures that the monitor_system function is called only when the script
# is executed directly (not when imported as a module).
if __name__ == "__main__":
    # Example usage: Monitor every 5 seconds for 1 minute.
    # To monitor indefinitely every 10 seconds, comment the line below
    # and uncomment the line after that.
    monitor_system(interval_seconds=5, duration_minutes=1)

    # Example usage: Monitor indefinitely every 10 seconds.
    # monitor_system(interval_seconds=10, duration_minutes=None)